In [2]:
from read_ASC import *
measurement = dls_sls_analysis('/home/thijssnel/programeren/jaar_4/technische_stage/NaCl - 0.085micrometer - per_5_graden')

plt.rcParams.update({
    "font.family": "Calibri",        # Lettertype
    "xtick.direction": "in",         # x-Ticks naar binnen
    "ytick.direction": "in",         # y-Ticks naar binnen
    "xtick.top": True,               # Ticks aan bovenkant ook
    "ytick.right": True,             # Ticks aan rechterkant ook
    "xtick.labelsize": 12,           # Labelgroottes gelijk
    "ytick.labelsize": 12,
    "axes.labelsize": 15,            # As-labels dezelfde grootte
    'svg.fonttype': 'none'           # inkscape dingetje
})

In [6]:
def zimm_plot_data(concentration: float):
    records = []

    for file, obj in measurement.get_data(data_type='solution').items():
        angle = round(obj.Angle_deg)
        total_countrate = float(np.mean(obj.CountRatey))
        duration = obj.Duration_s
        time = obj.Time
        q_vec = obj.q
        mon_diode = obj.MonitorDiode

        records.append({
            "angle_deg": angle,
            "duration_s": duration,
            "time": time,
            "sum_countrate": total_countrate,
            "monitor_diode": mon_diode
        })

    df_solution = pd.DataFrame(records)

    df_grouped_solution = df_solution.groupby('angle_deg')['sum_countrate']
    df_grouped_mean_solution = df_grouped_solution.mean()

    df_grouped_diode = df_solution.groupby('angle_deg')['monitor_diode']
    df_grouped_mean_diode_solution = df_grouped_diode.mean()

    # Constants
    dn_dc = 0.11 # verandering brekingsindex bij verschillende concentraties - variabele
    n0 = 1.481 # brekings index cis Decalin op kamer temperatuur - variabele 
    lambda0 = 632.8e-9  # golflengte in meter - constante
    N_A = 6.023e23 # Avogadros getal - constante
    r = 0.14           # afstand detector tegen cuvette in meter - constante
    V = 2.5e-6         # volume cuvette in kubieke meter - constante

    I_0 = df_grouped_mean_diode_solution.values
    df_solvent = measurement.get_data(data_type='solvent')

    records_solvent = []
    for __, obj in df_solvent.items():
        angle = round(obj.Angle_deg)
        total_countrate = float(np.mean(obj.CountRatey))
        records_solvent.append({
            "angle_deg": angle,
            "sum_countrate": total_countrate
        })

    df_solvent = pd.DataFrame(records_solvent)
    df_grouped_solvent = df_solvent.groupby('angle_deg')['sum_countrate']
    df_grouped_mean_solvent = df_grouped_solvent.mean()

    I_theta = df_grouped_mean_solution.values - df_grouped_mean_solvent.values

    theta = np.radians(df_grouped_mean_solution.index.values) # gemeten hoeken

    R_theta = (I_theta / I_0) * (r**2 / V)

    K = ((4*np.pi**2) / (lambda0**4 * N_A)) * (n0 * dn_dc)**2
    c = concentration

    x = np.sin(theta / 2)**2 # x-as Zimmplot
    y = (K * c) / R_theta # y-as Zimmplot
    return x, y

In [7]:
c_0 = zimm_plot_data(concentration=5.49e-1)
c_1 = zimm_plot_data(concentration=5.49e-2)
c_2 = zimm_plot_data(concentration=5.49e-3)
c_3 = zimm_plot_data(concentration=5.49e-4)
c_4 = zimm_plot_data(concentration=5.49e-5)
c_5 = zimm_plot_data(concentration=5.49e-6)

ValueError: operands could not be broadcast together with shapes (29,) (141,) 

In [5]:
plt.scatter(c_0[0], c_0[1], color='C0', label='c0')
plt.scatter(c_1[0], c_1[1], color='C1', label='c1')
plt.scatter(c_2[0], c_2[1], color='C2', label='c2')
plt.scatter(c_3[0], c_3[1], color='C3', label='c3')
plt.scatter(c_4[0], c_4[1], color='C4', label='c4')
plt.scatter(c_5[0], c_5[1], color='C5', label='c5')

plt.xlabel('sin(theta/2)**2')
plt.ylabel('KC/R_theta')
plt.grid()
plt.legend()
plt.tight_layout()
plt.yscale('log')
plt.show()

NameError: name 'c_0' is not defined

In [ ]:
def zimm_analysis_from_measurements(datasets, concentrations, x_max=0.4):
    """
    datasets : list of tuples, each entry is (x, y) returned by zimm_plot_data
    concentrations : array-like, concentrations corresponding to each dataset
    x_max : float, upper bound for Guinier regime
    
    -------
    
    results : dict
        {
            'Mw'       : molecular weight,
            'Rg'          : radius of gyration,
            'slope_theta' : slope of Kc/R_ttheta vs sin^2(theta/2),
            'intercepts'   : intercept at theta → 0,
            'fit_coeffs' : (slope, intercept)
        }
    """

    concentrations = np.asarray(concentrations)
    n_conc = len(datasets)

    if n_conc != len(concentrations):
        raise ValueError("Number of datasets must match number of concentrations")

    # ---- stack data ----
    x, _ = datasets[0]
    x = np.asarray(x)

    y_stack = []

    for i, (x, y) in enumerate(datasets):
        x = np.asarray(x)
        y = np.asarray(y)

        if not np.allclose(x, x, rtol=1e-6, atol=1e-12):
            raise ValueError(f"x-grid mismatch in dataset {i}")

        y_stack.append(y)

    y_stack = np.asarray(y_stack)

    intercepts = []
    slopes_theta = []

    # θ --> 0 extrapolation 
    for i, c in enumerate(concentrations):
        yi = y_stack[i]

        mask = (
            np.isfinite(x) &
            np.isfinite(yi) &
            (x > 0) &
            (yi > 0) &
            (x < x_max)
        )

        if np.sum(mask) < 5:
            raise ValueError(f"Insufficient Guinier points at c = {c}")

        m, b = np.polyfit(x[mask], yi[mask], 1)

        if b <= 0:
            raise ValueError(
                f"Non-positive intercept at c = {c}. "
                "Check solvent subtraction or Rayleigh normalization."
            )

        slopes_theta.append(m)
        intercepts.append(b)

    intercepts = np.asarray(intercepts)
    slopes_theta = np.asarray(slopes_theta)

    # c --> 0 extrapolation 
    slope_c, intercept_c = np.polyfit(concentrations, intercepts, 1)

    if intercept_c <= 0:
        raise ValueError("Mw diverges (c→0 intercept ≤ 0)")

    Mw = 1.0 / intercept_c
    A2 = slope_c / 2.0

    # radius of gyration
    # slope_theta = (1 / 3Mw) * Rg^2
    Rg2 = 3.0 * Mw * slopes_theta
    Rg2 = Rg2[Rg2 > 0]

    if len(Rg2) == 0:
        raise ValueError("All Rg² estimates are non-physical")

    Rg = np.sqrt(np.mean(Rg2))

    return {
        "Mw": Mw,
        "A2": A2,
        "Rg": Rg,
        "intercepts": intercepts,
        "slopes_theta": slopes_theta,
        "c_fit": (slope_c, intercept_c)
    }

In [11]:
datasets = [c_0, c_1, c_2, c_3, c_4, c_5]
concentrations = np.array([
    5.49e-1,
    5.49e-2,
    5.49e-3,
    5.49e-4,
    5.49e-5,
    5.49e-6
])

results = zimm_analysis_from_measurements(
    datasets=datasets,
    concentrations=concentrations,
    x_max=0.35
)

print(f"Mw = {results['Mw']:.3e} g/mol")
print(f"A2 = {results['A2']:.3e}")
print(f"Rg = {results['Rg']*1e9:.2f} nm")

ValueError: Mw diverges (c→0 intercept ≤ 0)

In [24]:
def zimm_single_concentration_analysis(x, y, x_max=0.4):
    """
    x : x-ais Zimmplot
    y : y-axs Zimmplot
    x_max : float cuttoff for Guinier region
    -------
    results : dict
        {
            'M_app'       : apparent molecular weight (same units as K⁻¹),
            'Rg'          : radius of gyration (same length units as q⁻¹),
            'slope_theta' : slope of Kc/Rθ vs sin^2(theta/2),
            'intercept'   : intercept at theta → 0,
            'fit_coeffs' : (slope, intercept)
        }
    """
    
    x = np.asarray(x)
    y = np.asarray(y)

    # Guard against NaNs and negatives
    mask = np.isfinite(x) & np.isfinite(y) & (x > 0) & (y > 0) & (x < x_max)
    x_fit = x[mask]
    y_fit = y[mask]

    if len(x_fit) < 5:
        raise ValueError("Not enough valid low-angle points for a stable Zimm fit.")

    slope_theta, intercept = np.polyfit(x_fit, y_fit, 1)

    if intercept <= 0:
        raise ValueError(
            "Intercept is non-positive. Rayleigh calibration or solvent subtraction is still incorrect."
        )

    # Apparent molecular weight
    M_app = 1.0 / intercept

    # Radius of gyration: slope = (1 / 3M) * Rg^2
    Rg2 = 3.0 * M_app * slope_theta
    if Rg2 <= 0:
        raise ValueError("Computed Rg^2 is non-positive. Check intensity normalization.")

    Rg = np.sqrt(Rg2)

    return {
        "Apparent_Mw": M_app,
        "Rg": Rg,
        "slope_theta": slope_theta,
        "intercept": intercept,
        "fit_coeffs": (slope_theta, intercept),
        "n_points": len(x_fit)
    }

In [25]:
results = zimm_single_concentration_analysis(x=c_0[0], y=c_0[1])
results

{'Apparent_Mw': np.float64(87.7919152850239),
 'Rg': np.float64(11.221347773462753),
 'slope_theta': np.float64(0.4780950707674009),
 'intercept': np.float64(0.011390570495624969),
 'fit_coeffs': (np.float64(0.4780950707674009),
  np.float64(0.011390570495624969)),
 'n_points': 56}